In [1]:
#import and download dataset from Kaggle API
import kaggle
!kaggle datasets download ahmedmohamed2003/retail-store-sales-dirty-for-data-cleaning

Dataset URL: https://www.kaggle.com/datasets/ahmedmohamed2003/retail-store-sales-dirty-for-data-cleaning
License(s): CC-BY-SA-4.0




  0%|          | 0.00/221k [00:00<?, ?B/s]
100%|##########| 221k/221k [00:00<00:00, 172MB/s]


In [2]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('retail-store-sales-dirty-for-data-cleaning.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file

In [58]:
#read data from the file
import pandas as pd
df = pd.read_csv('retail_store_sales.csv')

In [59]:
#show dataframe
df

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,NaN
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,False
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,NaN
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,True


In [60]:
#standardize columns
df.columns = df.columns.str.strip().str.lower().str.replace(" ","_")

In [61]:
#checking null values
df.isnull().sum()

transaction_id         0
customer_id            0
category               0
item                1213
price_per_unit       609
quantity             604
total_spent          604
payment_method         0
location               0
transaction_date       0
discount_applied    4199
dtype: int64

In [62]:
#handling null values
df = df.copy()   # make a full copy
df['price_per_unit'] = df['price_per_unit'].fillna(df['total_spent'] / df['quantity'])
df['total_spent'] = df['total_spent'].fillna(df['price_per_unit'] * df['quantity'])
df = df.dropna(subset=['quantity', 'total_spent'], how='all')
df['discount_applied'] = df['discount_applied'].map({True: 'Yes', False: 'No'})

In [63]:
# slice of another DataFrame
df['item'] = df.groupby(['category','price_per_unit'])['item'].transform('first')

In [64]:
df.dtypes

transaction_id       object
customer_id          object
category             object
item                 object
price_per_unit      float64
quantity            float64
total_spent         float64
payment_method       object
location             object
transaction_date     object
discount_applied     object
dtype: object

In [72]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'],format="%Y-%m-%d")
# keep only date part (drops time)
df['transaction_date'] = df['transaction_date'].dt.date

In [73]:
df

,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,Yes
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,Yes
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,No
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,No
...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,NaN
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,No
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,NaN
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,Yes


In [74]:
#sql server connection
import sqlalchemy as sal
engine = sal.create_engine("mssql+pyodbc://SONU/master?driver=ODBC Driver 17 for SQL Server")
conn = engine.connect()

In [75]:
#create new table in sql
df.to_sql('retail_orders',con=conn,index=False,if_exists = 'replace')

1